한국어 자연어처리의 전반적인 FLOW를 이해하고, 간단한 LSTM으로 영화리뷰 감성분석 모델을 훈련합니다

필요한 라이브러리를 import해줍니다.

* Embedding을 통해 단어를 벡터로 숫자로 바꿔주겠습니다.
* LSTM을 활용해 RNN 신경망을 구성합니다.
* Dense로 LSTM의 마지막 feature를 연결하여 긍정, 부정으로 분류합니다
* pad_sequences는 sequential data의 길이를 padding하거나 clipping하여 일괄 맞추는 전처리를 수행합니다








In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM  
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Step 0. 학습 데이터 준비하기
<img src = "https://github.com/seungyounglim/temporary/blob/master/image_5.PNG?raw=true">    

- 네이버 영화 감성분석 데이터셋 활용
- 훈련 데이터 150,000건, 테스트 데이터 50,000건

In [ ]:
""" 네이버 영화 리뷰 데이터셋 다운로드 """
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

""" 데이터 읽어오기 """
with open("ratings_train.txt") as f:
    raw_train = f.readlines()
with open("ratings_test.txt") as f:
    raw_test = f.readlines()
raw_train = [t.split('\t') for t in raw_train[1:]]
raw_test = [t.split('\t') for t in raw_test[1:]]

FULL_TRAIN = []
for line in raw_train:
    FULL_TRAIN.append([line[0], line[1], int(line[2].strip())])
FULL_TEST = []
for line in raw_test:
    FULL_TEST.append([line[0], line[1], int(line[2].strip())]) 

<img src = "https://github.com/seungyounglim/temporary/blob/master/image_6.PNG?raw=true">  
- 시간 관계상 train 중 50,000건을 학습데이터, 10,000건을 검증 데이터로 사용합니다.
- test 중 10,000건만 샘플링하여 최종 성능 테스트에 사용하겠습니다

In [ ]:
import random
random.seed(1)
random.shuffle(FULL_TRAIN)
random.shuffle(FULL_TEST)
train = FULL_TRAIN[:50000]
val = FULL_TRAIN[50000:60000]
test = FULL_TEST[:10000]
print("train     : {}개 (긍정 {}, 부정 {})".format(len(train), sum([t[2] for t in train]), len(train)-sum([t[2] for t in train])), train[0])
print("validation: {}개 (긍정 {}, 부정 {})".format(len(val), sum([t[2] for t in val]), len(val)-sum([t[2] for t in val])), val[0])
print("test      : {}개 (긍정 {}, 부정 {})".format(len(test), sum([t[2] for t in test]), len(test)-sum([t[2] for t in test])), test[0])

라벨을 보니 0이 부정 리뷰, 1이 긍정 리뷰입니다. 

한국말은 끝까지 읽어봐야 합니다.

### Step 1. Tokenizing(Parsing)
- 문장을 음절(character)단위로 쪼갭니다
- 예시 문장 두 개를 넣어서 어떻게 쪼개지는지 알 수 있습니다


In [ ]:
def tokenize(sentence): 
  return [char for char in sentence]

In [ ]:
tokenize("문장을 한 글자씩 쪼개줍니다.")

Train/ Test의 문장을 음절단위로 나눕니다

정답 라벨은 부정(0), 긍정(1)로 이루어져있습니다

In [ ]:
train_sentences = []
val_sentences = []
test_sentences = []
 
train_label_ids = []
val_label_ids = []
test_label_ids = []
 
for i, line in enumerate(train):
    words = tokenize(line[1])
    train_sentences.append(words) 
    train_label_ids.append(line[2])   

for line in val:
    words = tokenize(line[1])
    val_sentences.append(words) 
    val_label_ids.append(line[2])  
 
for line in test:
    words = tokenize(line[1])
    test_sentences.append(words) 
    test_label_ids.append(line[2])  

##Step 2. 모델 인풋 만들기

#### 2-1) 음절 사전 만들기
각 음절을 모델이 처리할 수 있는 정수 인덱스로 변환해야 합니다.
- 훈련 데이터 문장에 있는 음절을 정수로 매핑하는 사전을 만들고,
- 배치 연산을 위해 필요한 Padding([PAD])과 Out of vocabulary([OOV]) 토큰을 항상 맨 앞에 추가해줍니다

(일반적으로는 더 많은 코퍼스에 대해 구축된 사전을 사용하지만, 편의상 훈련셋만으로 진행합니다)

In [ ]:
from tqdm import tqdm

vocab_dict = {}
vocab_dict["[PAD]"] = 0
vocab_dict["[OOV]"] = 1
i = 2
for sentence in train_sentences:
    for word in sentence:
        if word not in vocab_dict.keys(): 
            vocab_dict[word] = i
            i += 1
print("Vocab Dictionary Size:", len(vocab_dict))

#### 2-2) vocab_dict를 이용해 자연어를 정수 인덱스로 바꾸기


#### 2-2) vocab_dict를 이용해 자연어를 정수 인덱스로 바꾸기
- 위에서 만든 vocab_dict를 이용해 잘라놓은 문장을 모델에 태울 수 있는 정수 인덱스로 바꾸어줍니다
 
    - 이 때, 사전에서 매핑되는 음절은 해당 인덱스로 바꾸고 사전에 없는 음절은 [OOV] 인덱스로 처리합니다.

- 기본적으로 LSTM은 가변적인 문장 길이를 인풋으로 받을 수 있지만, 배치 처리를 위해 <font color="blue">max_seq_len</font>을 정해두고 길이를 통일합니다.
    - max_seq_len 보다 짧은 문장에는 max_seq_len이 될 때까지 [PAD]에 해당하는 인덱스를 붙여줍니다
    - max_seq_len 보다 긴 문장은 max_seq_len 개의 토큰만 남기고 자릅니다

기본적으로 [PAD]는 시퀀스의 앞에 붙이는 것이 관례입니다. 앞부분이 상대적으로 RNN 모델이 잊어버릴 가능성이 크기 때문에, 쓸모없는 부분은 앞으로 몰아넣습니다. 

In [ ]:
def make_input_ids(tokenized_sentences, max_seq_len = 50):
  
  num_oov = 0 # OOV 발생 개수를 셈
  result_input_ids = [] # result_input_ids : 정수 인덱스로 변환한 문장들의 리스트

  for sentence in tokenized_sentences :
      """ vocab_dict를 사용해 정수로 변환 """ 
      input_ids = []
      for word in sentence:
          if word not in vocab_dict:   ## 사전에 없는 음절은 OOV 처리
              input_ids.append(vocab_dict['[OOV]']) 
              num_oov += 1
          else:                       ## 사전에 있는 음절은?
              input_ids.append(vocab_dict[word]) ##  vocab_dict 사전에서 토큰 찾아서 붙이기
      
      result_input_ids.append(input_ids)
      
  """ max_seq_len을 넘는 문장은 절단, 모자르는 것은 PADDING """
  result_input_ids = pad_sequences(result_input_ids, maxlen=max_seq_len, value=vocab_dict["[PAD]"]) ##  padding 하기

  return result_input_ids, num_oov


In [ ]:
# train_sentences 처리
train_input_ids, num_oov = make_input_ids(train_sentences)

print("---- TRAIN ----")
print("... # OOVs     :", num_oov)

In [ ]:
# val_sentences 처리
val_input_ids, num_oov = make_input_ids(val_sentences)

print("---- VALIDATION ----")
print("... # OOVs     :", num_oov)

In [ ]:
# test_sentences 처리
test_input_ids, num_oov = make_input_ids(test_sentences)

print("---- TEST ----")
print("... # OOVs     :", num_oov)

#### 2-3) 라벨 리스트를 np.array로 변환해줍니다.


In [ ]:
train_label_ids = np.array(train_label_ids)
val_label_ids = np.array(val_label_ids)
test_label_ids = np.array(test_label_ids)

## Step3. 모델 만들기


LSTM을 사용해 문장을 인코딩하고, Dense layer을 쌓아 최종 output을 생성합시다

# 실습 MISSION : RNN 모델 하이퍼파라미터 변경하기

여러 설정들을 마음대로 변경해봅시다.

- 임베딩 레이어 : 음절을 몇 차원의 벡터로 변경할지 조정합니다.
- LSTM : LSTM의 hidden size를 조정합니다.
- Dense : 추가, 삭제, 노드 수를 조정합니다.
- 기타 regularization 효과를 원하면 import하고 추가합니다
 

In [ ]:
vocab_size = len(vocab_dict)        # 단어사전 개수
embedding_dim = 64     # 임베딩 size
lstm_hidden_dim = 50   # LSTM hidden_size 
dense_dim = 64         # Dense layer size

model = Sequential([
    Embedding(vocab_size, embedding_dim),
    LSTM(lstm_hidden_dim),
    Dense(dense_dim, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

# Step 4. 모델 훈련하기

loss, optimizer를 지정하고 학습합니다.


In [ ]:
EPOCHS = 10
BATCHS = 128

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_input_ids, train_label_ids, epochs=EPOCHS, batch_size=BATCHS, validation_data=(val_input_ids, val_label_ids), verbose=2) 

test_result = model.evaluate(test_input_ids, test_label_ids, verbose=2)

학습 결과를 확인합니다.

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

원하는 문장으로 결과를 추론해봅니다.

스코어가 0에 가까울수록 부정, 1에 가까울수록 긍정입니다. 

In [ ]:
""" 학습된 모델로  예측해보기 """

def inference(mymodel, sentence):
  # 1. tokenizer로 문장 파싱
  words = tokenize(sentence)
  input_id = []

  # 2. vocab_dict를 이용해 인덱스로 변환
  for word in words:
    if word in vocab_dict: input_id.append(vocab_dict[word])
    else: input_id.append(vocab_dict["[OOV]"])
  
  # 단일 문장 추론이기 때문에 패딩할 필요가 없음 
  score = mymodel.predict(np.array([input_id])) 

  print("** INPUT:", sentence, end="")
  print(" ->  {:.2f}".format(score[0][0]))

In [ ]:
# 원하는 문장에 대해 추론해 보세요 
inference(model, "안보면 후회ㅠㅠ...")
inference(model, "이런 망작을 나 혼자만 보기엔 아깝지")
inference(model, "이런 꿀잼을 나 혼자만 보기엔 아깝지")
inference(model, "꿀잠 잤습니다")